In [36]:
from collections import defaultdict
from PIL import Image
from pathlib import Path
import imagehash
from tqdm import tqdm
import shutil

In [40]:
def find_similar_images(
    image_paths: list[Path],
    threshold: int = 5
) -> dict[str, list[Path]]:
    """
    Группирует похожие изображения на основе хеша.
    
    :param image_paths: список путей к изображениям
    :param threshold: максимальное расстояние между хешами (0-64)
                     чем меньше, тем строже сравнение
    :return: {representative_hash: [список похожих путей]}
    """
    hashes = {}
    groups = defaultdict(list)
    
    # Сначала собираем все хеши
    for path in tqdm(image_paths):
        try:
            img = Image.open(path)
            h = imagehash.phash(img)
            hashes[path] = h
        except Exception as e:
            print(f"Ошибка обработки {path}: {e}")
    
    # Группируем похожие изображения
    used_paths = set()
    for path1, hash1 in tqdm(hashes.items()):
        if path1 in used_paths:
            continue
            
        current_group = [path1]
        for path2, hash2 in hashes.items():
            if path2 not in used_paths and path1 != path2:
                if hash1 - hash2 <= threshold:  # Расстояние Хемминга
                    current_group.append(path2)
        
        if len(current_group) > 1:  # Только группы с похожими изображениями
            for p in current_group:
                used_paths.add(p)
            # Используем hex-представление хэша как ключ
            groups[str(hash1)].extend(current_group)
    
    return dict(groups)

In [43]:
def find_and_move_duplicates(
    input_dir: Path,
    output_dir: Path,
    threshold: int = 5,
    min_group_size: int = 2
) -> int:
    """
    Находит дубликаты в папке и перемещает их в подпапки output_dir.
    
    :param input_dir: Папка с изображениями для проверки
    :param output_dir: Папка для сохранения групп дубликатов
    :param threshold: Максимальное расстояние между хешами (0-64)
    :param min_group_size: Минимальный размер группы для перемещения
    """
    if not output_dir.exists():
        output_dir.mkdir(parents=True)
    
    # Находим похожие изображения
    groups = find_similar_images(sorted(input_dir.iterdir()), threshold)
    
    # Перемещаем группы дубликатов
    moved = 0
    for group_num, (hash_val, paths) in enumerate(groups.items(), 1):
        if len(paths) >= min_group_size:
            # Первый файл в группе считаем оригиналом (не перемещаем)
            original = paths[0]
            print(f"\nОригинал: {original.name}")
            
            filename_without_suffix = original.name.replace(original.suffix, '')
            group_dir = output_dir / f"duplicates_{filename_without_suffix}_{group_num}"
            group_dir.mkdir(exist_ok=True)
            
            # Перемещаем дубликаты
            for duplicate in paths[1:]:
                shutil.move(duplicate, group_dir / duplicate.name)
                # print(f"Дубликат → {group_dir.name}/{duplicate.name}")
                moved += 1
    
    return moved

In [45]:
current_dir = Path.cwd()
print(f"Директория: {current_dir}")
data_dir = current_dir / "data"
dataset_dir = data_dir / "cian_data"
# cian_data_dir = data_dir / "cian_data"
# output_data_dir = data_dir / "floor_plans"

for path in sorted(dataset_dir.iterdir()):
    if path.is_dir():
        moved = find_and_move_duplicates(input_dir=path, output_dir=path / "dublicates")
        print(f"{path.name} | Moved {moved}")

Директория: /home/little-garden/CodeProjects/InteriorClass


 41%|████▏     | 17533/42501 [00:25<00:35, 704.65it/s]

Ошибка обработки /home/little-garden/CodeProjects/InteriorClass/data/cian_data/A0/dublicates: [Errno 21] Is a directory: '/home/little-garden/CodeProjects/InteriorClass/data/cian_data/A0/dublicates'


100%|██████████| 42500/42500 [42:42<00:00, 16.59it/s] 



Оригинал: 1135068107-4.jpg

Оригинал: 1353458550-4.jpg

Оригинал: 1353458568-4.jpg

Оригинал: 1392677103-4.jpg

Оригинал: 1392677174-4.jpg

Оригинал: 1392677355-4.jpg

Оригинал: 1392677613-4.jpg

Оригинал: 1392677689-4.jpg

Оригинал: 1392677889-4.jpg

Оригинал: 1392677993-4.jpg

Оригинал: 1392678099-4.jpg

Оригинал: 1392678192-4.jpg

Оригинал: 1392678221-4.jpg

Оригинал: 1392678318-4.jpg

Оригинал: 1392678319-4.jpg

Оригинал: 1392678459-4.jpg

Оригинал: 1392678497-4.jpg

Оригинал: 1392678718-4.jpg

Оригинал: 1510660231-4.jpg

Оригинал: 1799457823-4.jpg

Оригинал: 1799457833-4.jpg

Оригинал: 1799457847-4.jpg

Оригинал: 1799457886-4.jpg

Оригинал: 1938586166-4.jpg

Оригинал: 1938586179-4.jpg

Оригинал: 1938586272-4.jpg

Оригинал: 2131624555-4.jpg

Оригинал: 2131624558-4.jpg

Оригинал: 2144450024-4.jpg

Оригинал: 2147012909-4.jpg

Оригинал: 2155684694-4.jpg

Оригинал: 2155684695-4.jpg

Оригинал: 2155684696-4.jpg

Оригинал: 2155684700-4.jpg

Оригинал: 2155684702-4.jpg

Оригинал: 215568470

...


Оригинал: 1458820008-4.jpg

Оригинал: 2339225290-4.jpg

Оригинал: 2339225306-4.jpg

Оригинал: 2339225313-4.jpg

Оригинал: 2350123155-4.jpg

Оригинал: 2350123163-4.jpg

Оригинал: 2350123169-4.jpg

Оригинал: 2350123174-4.jpg

Оригинал: 2354069500-4.jpg

Оригинал: 2354069504-4.jpg

Оригинал: 2354069511-4.jpg

Оригинал: 2379044821-4.jpg

Оригинал: 2379044828-4.jpg

Оригинал: 2379044835-4.jpg

Оригинал: 2379044854-4.jpg

Оригинал: 2379044859-4.jpg

Оригинал: 2379044867-4.jpg

Оригинал: 2386408288-4.jpg

Оригинал: 2388335780-4.jpg

Оригинал: 2427286224-4.jpg

Оригинал: 2427286233-4.jpg

Оригинал: 2437481573-4.jpg

Оригинал: 2439326052-4.jpg

Оригинал: 2439326059-4.jpg

Оригинал: 2439326087-4.jpg

Оригинал: 2439326092-4.jpg

Оригинал: 2444309129-4.jpg

Оригинал: 2444309554-4.jpg

Оригинал: 2448147868-4.jpg

Оригинал: 2448148209-4.jpg

Оригинал: 2455853477-4.jpg

Оригинал: 2455853508-4.jpg

Оригинал: 2462773359-4.jpg

Оригинал: 2462773729-4.jpg

Оригинал: 2462773759-4.jpg

Оригинал: 246315988

 20%|██        | 18617/92631 [00:27<01:44, 710.07it/s]

Ошибка обработки /home/little-garden/CodeProjects/InteriorClass/data/cian_data/D1_uncertain/dublicates: [Errno 21] Is a directory: '/home/little-garden/CodeProjects/InteriorClass/data/cian_data/D1_uncertain/dublicates'


100%|██████████| 92630/92630 [3:32:37<00:00,  7.26it/s]  



Оригинал: 1043668832-4.jpg

Оригинал: 1043668863-4.jpg

Оригинал: 1674839922-4.jpg

Оригинал: 1674839924-4.jpg

Оригинал: 1674839940-4.jpg

Оригинал: 1674839950-4.jpg

Оригинал: 1804131952-4.jpg

Оригинал: 2244847380-4.jpg

Оригинал: 2244847458-4.jpg

Оригинал: 2244847472-4.jpg

Оригинал: 2244847557-4.jpg

Оригинал: 2244847613-4.jpg

Оригинал: 2244847719-4.jpg

Оригинал: 2244847759-4.jpg

Оригинал: 2244847825-4.jpg

Оригинал: 2261078509-4.jpg

Оригинал: 2270956318-4.jpg

Оригинал: 2270956327-4.jpg

Оригинал: 2270956339-4.jpg

Оригинал: 2270956346-4.jpg

Оригинал: 2270956359-4.jpg

Оригинал: 2270956372-4.jpg

Оригинал: 2270956380-4.jpg

Оригинал: 2270956388-4.jpg

Оригинал: 2290321832-4.jpg

Оригинал: 2290321837-4.jpg

Оригинал: 2290321841-4.jpg

Оригинал: 2290321868-4.jpg

Оригинал: 2290321882-4.jpg

Оригинал: 2290321910-4.jpg

Оригинал: 2290321921-4.jpg

Оригинал: 2315476272-4.jpg

Оригинал: 2319312528-4.jpg

Оригинал: 2339225270-4.jpg

Оригинал: 2339225281-4.jpg

Оригинал: 234123338

In [33]:
groups

{'d3336cca4b9c9493': [PosixPath('/home/little-garden/CodeProjects/InteriorClass/data/yandex_data/A0/108.jpg'),
  PosixPath('/home/little-garden/CodeProjects/InteriorClass/data/yandex_data/A0/121.jpg')],
 'c0611f0efac13e7c': [PosixPath('/home/little-garden/CodeProjects/InteriorClass/data/yandex_data/A0/109.jpg'),
  PosixPath('/home/little-garden/CodeProjects/InteriorClass/data/yandex_data/A0/122.jpg')],
 '9794794e2e6a85d2': [PosixPath('/home/little-garden/CodeProjects/InteriorClass/data/yandex_data/A0/111.jpg'),
  PosixPath('/home/little-garden/CodeProjects/InteriorClass/data/yandex_data/A0/124.jpg')],
 'df205295c6d2e176': [PosixPath('/home/little-garden/CodeProjects/InteriorClass/data/yandex_data/A0/112.jpg'),
  PosixPath('/home/little-garden/CodeProjects/InteriorClass/data/yandex_data/A0/125.jpg')],
 'ff3f884ca7988831': [PosixPath('/home/little-garden/CodeProjects/InteriorClass/data/yandex_data/A0/113.jpg'),
  PosixPath('/home/little-garden/CodeProjects/InteriorClass/data/yandex_data/